In [2]:
# %%
# === IMPORTS ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
import joblib
import os

warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
sns.set_style("whitegrid")

print("✅ Imports OK")

# %%
# === CHARGEMENT DES DONNÉES ===
print("=" * 60)
print("CHARGEMENT DES DONNÉES")
print("=" * 60)

df = pd.read_csv("../data/interim/train_dataset_M1_interim.csv")

print(f"Dataset shape: {df.shape}")
print(f"Période Day: {df['Day'].min()} - {df['Day'].max()}")

# Vérifier Purchase
if "Purchase" in df.columns:
    print(f"\nTarget balance (Purchase):")
    print(df["Purchase"].value_counts(normalize=True))
else:
    print("\n⚠️ Colonne Purchase non trouvée (normal pour le test)")

print("\nPremières lignes:")
display(df.head())

# %%
# === SPLIT TRAIN/VAL (PAS DE TEST DANS CE FICHIER) ===
print("\n" + "=" * 60)
print("SPLIT TEMPOREL : TRAIN / VAL")
print("=" * 60)

train = df[df["Day"] <= 60].copy()
val = df[df["Day"] > 60].copy()

print(f"Train: Days 1-60   → {len(train)} lignes")
print(f"Val:   Days 61-{int(df['Day'].max())}  → {len(val)} lignes")

if "Purchase" in df.columns:
    print(f"\nDistribution Purchase:")
    print(f"  Train: {train['Purchase'].mean():.2%}")
    print(f"  Val:   {val['Purchase'].mean():.2%}")

# %%
# === FEATURE ENGINEERING - PARTIE 1 : Prix et Discount ===
print("\n" + "=" * 60)
print("FEATURE ENGINEERING - Prix & Discount")
print("=" * 60)

for data in [train, val]:
    data["Effective_Discount"] = data["Price"] * data["Discount"] / 100
    data["Net_Price"] = data["Price"] * (1 - data["Discount"] / 100)
    data["Price_Bucket"] = pd.qcut(data["Price"], 5, labels=False, duplicates="drop")

print("✅ Features prix créées")

# %%
# === FEATURE ENGINEERING - PARTIE 2 : Engagement & Interactions ===
print("\n" + "=" * 60)
print("FEATURE ENGINEERING - Engagement & Interactions")
print("=" * 60)

for data in [train, val]:
    data["Email_x_Engagement"] = data["Email_Interaction"] * data["Engagement_Score"]
    data["Cart_x_Engagement"] = data["Items_In_Cart"] * data["Engagement_Score"]

print("✅ Features engagement créées")

# %%
# === FEATURE ENGINEERING - PARTIE 3 : Segments EDA ===
print("\n" + "=" * 60)
print("FEATURE ENGINEERING - Segments EDA")
print("=" * 60)

for data in [train, val]:
    # Convertir Campaign_Period en bool
    data["Campaign_Period"] = data["Campaign_Period"].astype(bool)

    # Catégorie High Value
    data["HighValue_Category"] = data["Category"].isin([0.0, 1.0, 2.0]).astype(int)

    # Distance à la campagne
    def day_rel_to_campaign(day, campaign_windows=[(25, 50), (75, 90)]):
        if pd.isna(day):
            return np.nan
        campaign_days = [i for w in campaign_windows for i in range(w[0], w[1] + 1)]
        return min([abs(day - c) for c in campaign_days])

    data["Day_to_Campaign"] = data["Day"].apply(day_rel_to_campaign)

print("✅ Features segments créées")

# %%
# === 🚀 NOUVELLES FEATURES - INTERACTIONS CAMPAGNE ===
print("\n" + "=" * 60)
print("🚀 NOUVELLES FEATURES - INTERACTIONS CAMPAGNE")
print("=" * 60)

for data in [train, val]:
    # 1. Interactions multiplicatives campagne
    data["Price_x_Campaign"] = data["Price"] * data["Campaign_Period"].astype(int)
    data["Discount_x_Campaign"] = data["Discount"] * data["Campaign_Period"].astype(int)
    data["Net_Price_x_Campaign"] = data["Net_Price"] * data["Campaign_Period"].astype(
        int
    )

    # 2. Interactions engagement × campagne
    data["Engagement_x_Campaign"] = data["Engagement_Score"] * data[
        "Campaign_Period"
    ].astype(int)
    data["Items_x_Campaign"] = data["Items_In_Cart"] * data["Campaign_Period"].astype(
        int
    )
    data["Email_x_Campaign"] = data["Email_Interaction"] * data[
        "Campaign_Period"
    ].astype(int)

    # 3. Intensité campagne
    data["Campaign_Intensity"] = np.where(
        data["Campaign_Period"], 1 / (data["Day_to_Campaign"] + 1), 0
    )

    # 4. Ratios comportementaux
    data["Cart_Efficiency"] = data["Items_In_Cart"] / (data["Reviews_Read"] + 1)
    data["Engagement_per_Price"] = data["Engagement_Score"] / (data["Net_Price"] + 1)
    data["Email_Impact"] = (
        data["Email_Interaction"] * data["Engagement_Score"] * data["Reviews_Read"]
    )

    # 5. Segments avancés
    data["High_Intent"] = (
        (data["Items_In_Cart"] >= 3)
        & (data["Engagement_Score"] >= 2.5)
        & (data["Email_Interaction"] == 1)
    ).astype(int)

    data["Price_Sensitive"] = (
        (data["Discount"] >= 25) & (data["Net_Price"] <= 200)
    ).astype(int)

    data["Premium_Buyer"] = (
        (data["Net_Price"] >= 500) & (data["Device_Type"].isin(["Desktop", "Tablet"]))
    ).astype(int)

print(f"✅ 13 nouvelles features créées !")

# %%
# === MISSING INDICATORS ===
print("\n" + "=" * 60)
print("CRÉATION DES MISSING INDICATORS")
print("=" * 60)

missing_pct = train.isnull().sum() / len(train) * 100
cols_with_missing = missing_pct[missing_pct > 1].index.tolist()

# Exclure Day et Day_to_Campaign
cols_with_missing = [
    col for col in cols_with_missing if col not in ["Day", "Day_to_Campaign"]
]

for data in [train, val]:
    for col in cols_with_missing:
        data[f"{col}_missing"] = data[col].isnull().astype(int)

print(f"✅ {len(cols_with_missing)} missing indicators créés")
if cols_with_missing:
    print(f"   Colonnes: {cols_with_missing}")

# %%
# === IMPUTATION ===
print("\n" + "=" * 60)
print("IMPUTATION DES VALEURS MANQUANTES")
print("=" * 60)

# Calculer médianes sur train
median_values = {}
numeric_cols = train.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    if train[col].isnull().sum() > 0:
        median_values[col] = train[col].median()

# Appliquer sur train, val
for data in [train, val]:
    # Numériques
    for col in numeric_cols:
        if data[col].isnull().sum() > 0:
            fill_value = median_values.get(col, data[col].median())
            data[col].fillna(fill_value, inplace=True)

    # Catégorielles
    categorical_cols = data.select_dtypes(include=["object", "bool"]).columns
    for col in categorical_cols:
        if data[col].isnull().sum() > 0:
            data[col].fillna("Unknown", inplace=True)

print(f"✅ Imputation terminée")
print(f"   Train NA: {train.isnull().sum().sum()}")
print(f"   Val NA:   {val.isnull().sum().sum()}")

# %%
# === RARE CATEGORIES ===
print("\n" + "=" * 60)
print("GESTION DES CATÉGORIES RARES (<1%)")
print("=" * 60)

threshold = 0.01
categorical_cols = train.select_dtypes(include=["object"]).columns
rare_handling = {}

# Identifier sur train
for col in categorical_cols:
    if col == "Session_ID":
        continue
    value_counts = train[col].value_counts(normalize=True)
    rare_categories = value_counts[value_counts < threshold].index.tolist()

    if rare_categories:
        rare_handling[col] = rare_categories

# Appliquer sur train, val
for data in [train, val]:
    for col, rare_cats in rare_handling.items():
        if col in data.columns:
            data[col] = data[col].apply(lambda x: "Other" if x in rare_cats else x)

if rare_handling:
    print(f"✅ {len(rare_handling)} colonnes avec catégories rares regroupées")
else:
    print("✅ Aucune catégorie rare détectée")

# %%
# === 🧹 SUPPRESSION FEATURES INUTILES ===
print("\n" + "=" * 60)
print("🧹 NETTOYAGE - SUPPRESSION FEATURES INUTILES")
print("=" * 60)

features_to_remove = ["Effective_Discount"]

features_removed = []
for data in [train, val]:
    for feat in features_to_remove:
        if feat in data.columns:
            data.drop(columns=[feat], inplace=True)
            if feat not in features_removed:
                features_removed.append(feat)

print(f"✅ {len(features_removed)} features supprimées:")
for feat in features_removed:
    print(f"   - {feat}")

print(f"\n📊 Features totales: {len(train.columns)}")

# %%
# === PIPELINE CATBOOST ===
print("\n" + "=" * 60)
print("PRÉPARATION PIPELINE CATBOOST")
print("=" * 60)

cat_features = [
    "Device_Type",
    "Time_of_Day",
    "Payment_Method",
    "Referral_Source",
    "Category",
]

# Convertir en string
for data in [train, val]:
    for col in cat_features:
        if col in data.columns:
            data[col] = data[col].astype(str)

# Préparer données
cols_to_drop_cb = ["Session_ID", "Day", "Purchase", "id"]

X_train = train.drop(columns=cols_to_drop_cb, errors="ignore")
y_train = train["Purchase"] if "Purchase" in train.columns else None

X_val = val.drop(columns=cols_to_drop_cb, errors="ignore")
y_val = val["Purchase"] if "Purchase" in val.columns else None

# Indices catégoriels
cat_indices = [
    X_train.columns.get_loc(col) for col in cat_features if col in X_train.columns
]

print(f"✅ Pipeline CatBoost préparé:")
print(f"   X_train: {X_train.shape}")
print(f"   X_val:   {X_val.shape}")
print(f"   Categorical indices: {cat_indices}")

# %%
# === PIPELINE CLASSIC ===
print("\n" + "=" * 60)
print("PRÉPARATION PIPELINE CLASSIC (One-Hot + Scaling)")
print("=" * 60)

# Copier données
train_classic = train.copy()
val_classic = val.copy()

# Colonnes à drop
cols_to_drop = ["Session_ID", "Day", "Purchase", "id"]

for data in [train_classic, val_classic]:
    # Convertir Campaign_Period en int
    data["Campaign_Period"] = data["Campaign_Period"].astype(int)

# One-hot encoding
cols_onehot = ["Device_Type", "Time_of_Day", "Category"]
train_classic = pd.get_dummies(train_classic, columns=cols_onehot, drop_first=True)
val_classic = pd.get_dummies(val_classic, columns=cols_onehot, drop_first=True)

# Aligner les colonnes val sur train
missing_cols = set(train_classic.columns) - set(val_classic.columns)
for col in missing_cols:
    val_classic[col] = 0

extra_cols = set(val_classic.columns) - set(train_classic.columns)
val_classic = val_classic.drop(columns=list(extra_cols))

val_classic = val_classic[train_classic.columns]

# Label encoding
for col in ["Payment_Method", "Referral_Source"]:
    if col in train_classic.columns:
        le = LabelEncoder()
        train_classic[col] = le.fit_transform(train_classic[col].astype(str))
        val_classic[col] = le.transform(val_classic[col].astype(str))

# Préparer X, y
X_train_cls = train_classic.drop(columns=cols_to_drop, errors="ignore")
y_train_cls = train_classic["Purchase"] if "Purchase" in train_classic.columns else None

X_val_cls = val_classic.drop(columns=cols_to_drop, errors="ignore")
y_val_cls = val_classic["Purchase"] if "Purchase" in val_classic.columns else None

# Scaling
scaler = StandardScaler()
X_train_cls_scaled = scaler.fit_transform(X_train_cls)
X_val_cls_scaled = scaler.transform(X_val_cls)

X_train_cls = pd.DataFrame(X_train_cls_scaled, columns=X_train_cls.columns)
X_val_cls = pd.DataFrame(X_val_cls_scaled, columns=X_val_cls.columns)

print(f"✅ Pipeline Classic préparé:")
print(f"   X_train: {X_train_cls.shape}")
print(f"   X_val:   {X_val_cls.shape}")

# %%
# === VÉRIFICATION FINALE ===
print("\n" + "=" * 60)
print("VÉRIFICATION FINALE")
print("=" * 60)

print(f"✅ CatBoost:")
print(f"   Train: {X_train.shape}")
print(f"   Val:   {X_val.shape}")
print(f"   Features: {X_train.shape[1]}")

print(f"\n✅ Classic:")
print(f"   Train: {X_train_cls.shape}")
print(f"   Val:   {X_val_cls.shape}")
print(f"   Features: {X_train_cls.shape[1]}")

# %%
# === SAUVEGARDE ===
print("\n" + "=" * 60)
print("SAUVEGARDE DES DONNÉES PREPROCESSED")
print("=" * 60)

os.makedirs("../data/processed", exist_ok=True)

# CatBoost
catboost_data = {
    "X_train": X_train,
    "y_train": y_train,
    "X_val": X_val,
    "y_val": y_val,
    "cat_indices": cat_indices,
    "cat_features_names": [col for col in X_train.columns if col in cat_features],
}

joblib.dump(catboost_data, "../data/processed/catboost_ready.pkl", protocol=4)
print("✅ CatBoost: 'catboost_ready.pkl'")

# Classic
classic_data = {
    "X_train": X_train_cls,
    "y_train": y_train_cls,
    "X_val": X_val_cls,
    "y_val": y_val_cls,
    "scaler": scaler,
}

joblib.dump(classic_data, "../data/processed/classic_ready.pkl", protocol=4)
print("✅ Classic: 'classic_ready.pkl'")

# Résumé
print("\n" + "=" * 60)
print("RÉSUMÉ FINAL")
print("=" * 60)
print(f"📦 CatBoost: {X_train.shape[1]} features")
print(f"📦 Classic: {X_train_cls.shape[1]} features")
print(f"\n🚀 NOUVELLES FEATURES AJOUTÉES: +13 features")
print("\n🎉 PREPROCESSING TERMINÉ !")
print("=" * 60)

✅ Imports OK
CHARGEMENT DES DONNÉES
Dataset shape: (13735, 21)
Période Day: 1 - 70

Target balance (Purchase):
Purchase
0    0.631889
1    0.368111
Name: proportion, dtype: float64

Premières lignes:


,id,Age,Gender,Reviews_Read,Price,Discount,Category,Items_In_Cart,Time_of_Day,Email_Interaction,Device_Type,Payment_Method,Referral_Source,Socioeconomic_Status_Score,Engagement_Score,AB_Bucket,Price_Sine,Session_ID,Day,Campaign_Period,Purchase
0,1,NaN,1.0,3.0,592.975,22.0,1.0,6.0,afternoon,0.0,Mobile,Credit,Social_media,7.26,1.856520,3.0,0.999047,S0000003,59,False,0
1,2,25.0,1.0,1.0,511.279,12.0,0.0,3.0,morning,1.0,Tablet,Cash,Social_media,8.30,1.868138,5.0,-0.129689,S0000005,29,True,1
2,3,22.0,0.0,3.0,218.360,2.0,1.0,4.0,evening,1.0,Mobile,Bank,Social_media,6.61,1.223445,0.0,-0.421646,S0000007,16,False,0
3,4,24.0,0.0,3.0,313.781,1.0,3.0,0.0,evening,1.0,Mobile,PayPal,Social_media,10.51,0.359684,1.0,-0.988239,S0000008,53,False,0
4,5,35.0,1.0,1.0,495.088,13.0,0.0,2.0,evening,0.0,Mobile,Cash,Social_media,8.33,3.848580,2.0,0.695737,S0000009,10,False,0



SPLIT TEMPOREL : TRAIN / VAL
Train: Days 1-60   → 11766 lignes
Val:   Days 61-70  → 1969 lignes

Distribution Purchase:
  Train: 37.44%
  Val:   33.06%

FEATURE ENGINEERING - Prix & Discount
✅ Features prix créées

FEATURE ENGINEERING - Engagement & Interactions
✅ Features engagement créées

FEATURE ENGINEERING - Segments EDA
✅ Features segments créées

🚀 NOUVELLES FEATURES - INTERACTIONS CAMPAGNE
✅ 13 nouvelles features créées !

CRÉATION DES MISSING INDICATORS
✅ 29 missing indicators créés
   Colonnes: ['Age', 'Gender', 'Reviews_Read', 'Price', 'Discount', 'Category', 'Items_In_Cart', 'Time_of_Day', 'Email_Interaction', 'Device_Type', 'Socioeconomic_Status_Score', 'Engagement_Score', 'AB_Bucket', 'Price_Sine', 'Session_ID', 'Effective_Discount', 'Net_Price', 'Price_Bucket', 'Email_x_Engagement', 'Cart_x_Engagement', 'Price_x_Campaign', 'Discount_x_Campaign', 'Net_Price_x_Campaign', 'Engagement_x_Campaign', 'Items_x_Campaign', 'Email_x_Campaign', 'Cart_Efficiency', 'Engagement_per_Pr